In [ ]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties # for unicode fonts
import sys
import datetime as dt
import mp_utils as mp

from collections import OrderedDict

# used to print out pretty pandas dataframes
from IPython.display import display, HTML

from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# used to impute mean for data and standardize for computational stability
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# logistic regression is our favourite model ever
from sklearn import linear_model
from sklearn import ensemble

# used to calculate AUROC/accuracy
from sklearn import metrics

# used to create confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

# gradient boosting - must download package https://github.com/dmlc/xgboost
# originally used xgboost 0.6 - had to replace with 0.6a2
import xgboost as xgb

#from eli5 import show_weights

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']

%matplotlib inline

In [ ]:
df = pd.read_csv('X_design_matrix.csv').set_index('icustay_id')

# create X by dropping idxK and the outcome
y = df['death'].values
idxK = df['idxK']
X = df.drop(['death','idxK'],axis=1).values

print('{} observations. Outcome rate: {:2.2f}%.'.format(X.shape[0],
                                                        100.0*np.mean(y)))

    colsample_bytree     - 0.7 	[ 0.7 0.7 0.7 0.7 0.7 ]
    silent               - 1.0 	[ 1 1 1 1 1 ]
    learning_rate        - 0.01 	[ 0.01 0.01 0.01 0.01 0.01 ]
    n_estimators         - 1000.0 	[ 1000 1000 1000 1000 1000 ]
    subsample            - 0.8 	[ 0.8 0.8 0.8 0.8 0.8 ]
    objective            - skipping as it is a string
    max_depth            - 9.0 	[ 9 6 9 9 6 ]

In [ ]:
# Rough timing info:
#     rf - 3 seconds per fold
#    xgb - 30 seconds per fold
# logreg - 4 seconds per fold
#  lasso - 8 seconds per fold
np.random.seed(7390984)

# parameters from grid search
xgb_mdl = xgb.XGBClassifier(colsample_bytree=0.7, silent=1,
                            learning_rate = 0.01, n_estimators=1000,
                            subsample=0.8, max_depth=9)

models = {'xgb': xgb_mdl,
          'lasso': linear_model.LassoCV(cv=5,fit_intercept=True,normalize=True,max_iter=10000),
          'logreg': linear_model.LogisticRegression(fit_intercept=True),
          'l2': linear_model.LogisticRegressionCV()
          #'rf': ensemble.RandomForestClassifier()
         }

mdl_val = dict()
results_val = dict()
pred_val = dict()
pred_val_merged = dict()
for mdl in models:
    print('=============== {} ==============='.format(mdl))
    mdl_val[mdl] = list()
    results_val[mdl] = list() # initialize list for scores
    pred_val[mdl] = dict()
    pred_val_merged[mdl] = np.zeros(X.shape[0])
    
    if mdl == 'xgb':
        # no pre-processing of data necessary for xgb
        estimator = Pipeline([(mdl, models[mdl])])

    else:
        estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("scaler", StandardScaler()),
                      (mdl, models[mdl])]) 

    for k in range(K):
        # train the model using all but the kth fold
        curr_mdl = sklearn.base.clone(estimator).fit(X[idxK != k, :], y[idxK != k])

        # get prediction on this dataset
        if mdl in ('lasso','ridge'):
            curr_prob = curr_mdl.predict(X[idxK == k, :])
        else:
            curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
            curr_prob = curr_prob[:,1]
            
        pred_val_merged[mdl][idxK==k] = curr_prob
        pred_val[mdl][k] = curr_prob

        # calculate score (AUROC)
        curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

        # add score to list of scores
        results_val[mdl].append(curr_score)

        # save the current model
        mdl_val[mdl].append(curr_mdl)
        
        print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))
    
tar_val = dict()
for k in range(K):
    tar_val[k] = y[idxK==k]

In [ ]:
# average AUROC + min/max
for mdl in models:
    curr_score = np.zeros(K)
    for k in range(K):
        curr_score[k] = metrics.roc_auc_score(tar_val[k], pred_val[mdl][k])
    print('{}\t{:0.3f} [{:0.3f}, {:0.3f}]'.format(mdl, np.mean(curr_score), np.min(curr_score), np.max(curr_score)))

In [ ]:

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

In [ ]:
# plt AUROC 
models_pretty = OrderedDict()
models_pretty['xgb'] = 'Gradient Boosting'
models_pretty['logreg'] = 'Logistic regression'
models_pretty['lasso'] = 'L1-penalized regression'
models_pretty['l2'] = 'L2-penalized regression'

plt.figure()
i=0
for mdl in models_pretty:
    for k in range(K):
        curr_score = metrics.roc_auc_score(tar_val[k], pred_val[mdl][k])     
        plt.plot(curr_score, i, marker='o', color = col[i], markersize=16)
    i+=1
    
ax = plt.gca()
ax.set_yticks(range(i))
ax.set_yticklabels([models_pretty[x] for x in models_pretty])
ax.set_xlim([0.85, 0.95])
ax.set_xticks([0.85, 0.90, 0.95])
plt.show()

In [ ]:
# average AUPRC + min/max
for mdl in models:
    curr_score = np.zeros(K)
    for k in range(K):
        curr_score[k] = metrics.average_precision_score(tar_val[k], pred_val[mdl][k])
    print('{}\t{:0.3f} [{:0.3f}, {:0.3f}]'.format(mdl, np.mean(curr_score), np.min(curr_score), np.max(curr_score)))

# Plot model performance closer to death

In [ ]:
# load in design matrices
X_dict = {}
idxK_dict = {}
y_dict = {}

for t in [0, 4, 8, 16, 24]:
    df = pd.read_csv('X_design_matrix_{}hr_death.csv'.format(t)).set_index('icustay_id')

    # create X by dropping idxK and the outcome
    dm_string = 'Td={:02d}'.format(t)
    y_dict[dm_string] = df['death'].values
    idxK_dict[dm_string] = df['idxK']
    X_dict[dm_string] = df.drop(['death','idxK'],axis=1).values

    print('{:02d} - {} observations. Outcome rate: {:2.2f}%.'.format(t, X_dict[dm_string].shape[0],
                                                            100.0*np.mean(y_dict[dm_string])))

In [ ]:
# get predictions
score_pred = dict()
for m, mdl in enumerate(models):
    all_score = list()
    
    score_pred[mdl] = dict()
    for i, x in enumerate(X_dict):
        curr_score = list()
        for k in range(K):
            curr_mdl = mdl_val[mdl][k]

            # get prediction on this dataset
            if mdl in ('lasso','ridge'):
                curr_prob = curr_mdl.predict(X_dict[x][idxK_dict[x] == k, :])
            else:
                curr_prob = curr_mdl.predict_proba(X_dict[x][idxK_dict[x] == k, :])
                curr_prob = curr_prob[:,1]

            # calculate score (AUROC)
            curr_score.append(metrics.roc_auc_score(y_dict[x][idxK_dict[x] == k], curr_prob))
        score_pred[mdl][x] = curr_score

In [ ]:
models_pretty = OrderedDict()
models_pretty['xgb'] = 'Gradient Boosting'
models_pretty['logreg'] = 'Logistic regression'
models_pretty['lasso'] = 'L1-penalized regression'
models_pretty['l2'] = 'L2-penalized regression'

mdl_dict = models_pretty

In [ ]:
plt.rcParams.update({'font.size': 22})

In [ ]:
# plot a figure of the results
marker = ['o','s','v','d']
xi_str = ['Td=04','Td=08','Td=16','Td=24']
xi = [int(x[-2:]) for x in xi_str]

plt.figure(figsize=[12,8])
for m, mdl in enumerate(mdl_dict):
    all_score = list()
    for i, x in enumerate(xi_str):
        curr_score = score_pred[mdl][x]
        plt.plot(xi[i] * np.ones(len(curr_score)), curr_score,
                marker=marker[m], color=col[m],
                markersize=10, linewidth=2, linestyle=':',
                 label="_nolegend_")

        all_score.append(np.median(curr_score))
        
    # plot a line through the mean across all evaluations
    plt.plot(xi, all_score,
            marker=marker[m], color=col[m],
            markersize=10, linewidth=2, linestyle='-',
            label=mdl_dict[mdl])

plt.gca().set_ylim([0.85, 1.0])
plt.gca().set_xticks(np.linspace(0,24,7))
plt.gca().set_xlim([-1,25])
plt.gca().invert_xaxis()
plt.legend(loc='lower right',fontsize=16)
plt.xlabel('Lead time (hours)',fontsize=18)
plt.ylabel('AUROC',fontsize=18)

ax = plt.gca()

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 

plt.savefig('experimentb_auroc_over_time.png')
plt.show()

## Patient trends

In [ ]:
iids = [202887, 203461, 207012, 207046]

col_map = {202887: col[1],
          203461: col[2],
          207012: col[0],
          207046: col[3]}

plt.figure(figsize=[12,8])

for i, iid in enumerate(iids):
    # load in a single individual's data from saved csv files (for convenience)
    df_curr = pd.read_csv('./data/' + str(iid) + '.0.csv')
    df_curr.set_index('charttime_elapsed',inplace=True)

    # this can take ~10-30 seconds to generate the features for all the rows
    df_x = mp.extract_feature_across_sp(df_curr)
    X_curr = df_x.values.astype(float)

    # add in the static variables
    X_static = np.tile(df_static.loc[iid,vars_static].values, [X_curr.shape[0],1]).astype(float)
    X_curr = np.column_stack([X_curr, X_static])

    # create the outcome
    y_curr = co.loc[iid,'hospital_expire_flag']

    # generate probability for the window selected for this patient
    y_prob = xgb_model.predict_proba(X_curr)
    y_prob = y_prob[:,1]

    # plot this probability over time
    t = df_x.index.values/60.0/60.0 + 4.0 # convert to hours, add offset so t indexes the *end* time


    #df_x.head(n=20)
    if y_curr == 0:
        lbl = 'ID {} - survived'.format(iid)
    else:
        lbl = 'ID {} - expired'.format(iid)
        
    plt.plot(t,y_prob,'-',color=col_map[iid], linestyle='-', marker='o', linewidth=4, label=lbl)


plt.xlabel('Time since ICU admission (hours)',fontsize=18)
plt.ylabel('Risk of mortality',fontsize=18)
plt.legend(loc='upper left',fontsize=16)

ax = plt.gca()
ax.set_ylim([0,1])

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 

plt.savefig('example_risks.pdf')
plt.savefig('example_risks.png')
plt.show()